In [17]:
import os
import re
import pandas as pd
import regex
from library import *

In [19]:
def trac1_dataset_preprocess():
    df1 = load_dataset("agr_hi_train.csv")
    df2 = load_dataset("agr_en_train.csv")
    df = pd.concat([df1,df2])
    df = preprocess_text(df)
    df["message"].fillna('',inplace=True)
    df1 = load_dataset("agr_hi_dev.csv")
    df2 = load_dataset("agr_en_dev.csv")
    val_df = pd.concat([df1,df2])
    val_df = preprocess_text(val_df)
    y_train = df["class"]
    y_test = val_df["class"]
    x_train = df["message"]
    x_test = val_df["message"]
    return x_train,x_test,y_train,y_test
x_train,x_test,y_train,y_test = trac1_dataset_preprocess()

In [7]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 MB 8.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 90.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 102.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 102.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 95.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 78.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 30.5 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.7-py3-none-any.whl size=1438262 sha256=28f99958c79e53a422184d0bb8546470950059e5c95851aa8e021ba442a80de3
  Stored in directory: /home/ubuntu/.cache/pip/wheels/cd/de/72/6c0e59e93fcc7702ab33cc2a49fa726b5684222c8ed7171183
Successfully built jax
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 2.0.0
    Uninstalling importlib-metadata-2.0.0:
      Successfully uninstalled importlib-metadata-2.0.0


In [20]:
import json
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Concatenate, Dropout,GlobalMaxPooling1D,concatenate,Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [21]:
def get_max_text_len(msgs):
    return max( list( map( lambda msg : len(msg) , msgs ) ) )

def convert_classes_to_nums(y_train,y_test):
    classes = y_train
    le = LabelEncoder()
    integer_labels = le.fit_transform(classes)
    y_train = integer_labels
    y_test = le.fit_transform(y_test)
    return y_train,y_test,le


In [22]:
# Define the path to the pre-trained GloVe model file
glove_file = '../glove.twitter.27B.100d.txt'

# Define the format of the pre-trained GloVe model file
def read_glove_file(glove_file):
    with open(glove_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    word_to_vec = {}
    for line in lines:
        line = line.strip().split()
        word = line[0]
        vec = np.array([float(val) for val in line[1:]])
        word_to_vec[word] = vec
    return word_to_vec

# Load the pre-trained GloVe model
word_to_vec_map = read_glove_file(glove_file)

glove_embed_dict = word_to_vec_map

In [67]:
embed_dict = glove_embed_dict
text_data = x_train
val_text_data = x_test
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(text_data)
val_sequences = tokenizer.texts_to_sequences(val_text_data)
# Pad sequences to a fixed length
max_len = get_max_text_len(x_train) # Set the maximum sequence length
data = pad_sequences(sequences, maxlen=max_len)
val_data = pad_sequences(val_sequences, maxlen=max_len)

# Convert labels to one-hot encoding
y_train,y_test,le = convert_classes_to_nums(y_train,y_test)
labels = to_categorical(y_train)
test_labels = to_categorical(y_test)

#embedding matrix
embedding_dim = len(list(embed_dict.values())[0])
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word.lower() in embed_dict:
        embedding_matrix[i] = np.array(embed_dict[word.lower()])

embedding_layer = Embedding(num_words, embedding_dim, weights=[embedding_matrix], trainable=True)

x_train,y_train = data,labels
x_test,y_test = val_data,test_labels


# Define hyperparameters
embed_dim = 100
num_filters = 64
filter_sizes = [2, 3, 4]
dropout_rate = 0.5
batch_size = 64
epochs = 10


# Define input layer
input_layer = Input(shape=(max_len,))

# Add embedding layer
embedding = embedding_layer(input_layer)

# Add parallel convolutional layers with max pooling and global max pooling
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(embedding)
    pool_layer = MaxPooling1D(pool_size=max_len - filter_size + 1)(conv_layer)
    conv_layers.append(GlobalMaxPooling1D()(pool_layer))
concat_layer = concatenate(conv_layers, axis=1)

# Add dropout layer
dropout_layer = Dropout(dropout_rate)(concat_layer)

# Add output layer
output_layer = Dense(3, activation='softmax')(dropout_layer)

# Define model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile model with binary cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

# Train model with specified batch size and number of epochs
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))



Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 6367)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 6367, 100)    5938600     ['input_5[0][0]']                
                                                                                                  
 conv1d_12 (Conv1D)             (None, 6366, 64)     12864       ['embedding_4[0][0]']            
                                                                                                  
 conv1d_13 (Conv1D)             (None, 6365, 64)     19264       ['embedding_4[0][0]']            
                                                                                            

In [24]:
from tensorflow.keras.models import save_model, load_model

In [55]:
# Save the model
save_model(model, 'glove_trained_cnn1d_categ_cross_loss.h5')

In [68]:
# Load the model back
loaded_model = load_model('glove_trained_cnn1d_categ_cross_loss.h5')

In [41]:
def predict_class(model, tokenizer, max_len, label_encoder, text):
    # Tokenize the input text
    text_sequence = tokenizer.texts_to_sequences([text])
    # Pad the sequence to the maximum length
    padded_sequence = pad_sequences(text_sequence, maxlen=max_len)
    # Make the prediction
    prediction = model.predict(padded_sequence)[0]
    # Convert the prediction to the actual label
    predicted_label = np.argmax(prediction)
    predicted_class = label_encoder.inverse_transform([predicted_label])[0]
    return predicted_class
